In [1]:
import importlib
import os, sys
from urllib.request import urlretrieve

# setup ngrok server
server_args = []
# start a single meshcat server instance to use for remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np
from IPython.display import display, HTML
from pydrake.examples.manipulation_station import ManipulationStation

import matplotlib.pyplot as plt

import pydrake
from pydrake.all import (
    DiagramBuilder, ConnectMeshcatVisualizer, Simulator, FindResourceOrThrow,
    Parser, MultibodyPlant, RigidTransform, LeafSystem, BasicVector,
    JacobianWrtVariable, RollPitchYaw, SignalLogger, AddTriad,
    PiecewisePolynomial, PiecewiseQuaternionSlerp, RotationMatrix, Solve,
    TrajectorySource, ConstantVectorSource
)
from pydrake.multibody import inverse_kinematics
from pydrake.all import SnoptSolver, IpoptSolver
from pydrake.trajectories import PiecewisePolynomial

def setup_manipulation_station():
  builder = DiagramBuilder()
  station = builder.AddSystem(ManipulationStation(time_step=1e-3))
  station.SetupNutStation()
  station.Finalize()

  frames_to_draw = {"gripper": {"body"}}
  meshcat = ConnectMeshcatVisualizer(builder,
          station.get_scene_graph(),
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=False,
          frames_to_draw=frames_to_draw,
          zmq_url=zmq_url)

  wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
  builder.Connect(wsg_position.get_output_port(), station.GetInputPort("wsg_position"))

  diagram = builder.Build()

  plant = station.get_multibody_plant()
  context = plant.CreateDefaultContext()
  gripper = plant.GetBodyByName("body")

  initial_pose = plant.EvalBodyPoseInWorld(context, gripper)

  simulator = Simulator(diagram)
  simulator.set_target_realtime_rate(1.0)
  simulator.AdvanceTo(0.01)

  return initial_pose, meshcat

# Get initial pose of the gripper by using default context of manip station.
initial_pose, meshcat = setup_manipulation_station()

# Helper function to visualize triad given a rigid transform. 
def visualize_transform(name, transform, prefix='', length=0.15, radius=0.006):
  # Support RigidTransform as well as 4x4 homogeneous matrix.
  if isinstance(transform, RigidTransform):
    transform = transform.GetAsMatrix4()
  AddTriad(meshcat.vis, name=name, prefix=prefix, length=length, radius=0.005, opacity=0.2)
  meshcat.vis[prefix][name].set_transform(transform)

p_WR = np.array([0.7477, -0.1445, 0.4148]) # frame R: center of left door. 

p_Rhandle = np.array([-0.033, 0.1245, 0]) # handle: frame attached to right handle.
p_Whandle = p_WR + p_Rhandle 

p_Rhinge = np.array([0.008, -0.1395, 0]) # hinge: frame attached to right hinge.
p_Whinge = p_WR + p_Rhinge 

p_Rhinge_handle = p_Rhandle - p_Rhinge
r_Rhinge_handle = np.linalg.norm(p_Rhandle - p_Rhinge) # distance between handle and hinge.

theta_Rhinge_handle = np.arctan2(p_Rhinge_handle[1], p_Rhinge_handle[0])
angle_end = np.pi # end of angle. Decrease to 120~160 deg for the easy version.

# Interpolate pose for opening doors. 
def InterpolatePoseOpen(t):
  # Start by interpolating the yaw angle of the hinge.
  angle_start = theta_Rhinge_handle
  theta = angle_start + (angle_end - angle_start) * t
  # Convert to position and rotation.
  p_Whandle = r_Rhinge_handle * np.array([np.cos(theta), np.sin(theta), 0]) + p_Whinge
  # Add some offset here to account for gripper yaw angle. 
  R_Whandle = RollPitchYaw(0, 0, theta).ToRotationMatrix()
  X_Whandle = RigidTransform(R_Whandle, p_Whandle)

  # Add a little offset to account for gripper.
  p_handleG = np.array([0., 0.1, 0.])
  R_handleG = RollPitchYaw(0, np.pi, np.pi).ToRotationMatrix()
  X_handleG = RigidTransform(R_handleG, p_handleG)
  X_WG = X_Whandle.multiply(X_handleG)
  return X_WG

## Interpolate Pose for entry.
def make_gripper_orientation_trajectory():
  traj = PiecewiseQuaternionSlerp()
  traj.Append(0.0, initial_pose.rotation())
  traj.Append(5.0, InterpolatePoseOpen(0.0).rotation())
  return traj 

def make_gripper_position_trajectory():
  traj = PiecewisePolynomial.FirstOrderHold(
      [0.0, 5.0], 
      np.vstack([[initial_pose.translation()],
                 [InterpolatePoseOpen(0.0).translation()]]).T)
  return traj

entry_traj_rotation = make_gripper_orientation_trajectory()
entry_traj_translation = make_gripper_position_trajectory()

def InterpolatePoseEntry(t):
  return RigidTransform(RotationMatrix(entry_traj_rotation.value(t)), 
                        entry_traj_translation.value(t))

# Wrapper function for end-effector pose. Total time: 11 seconds.
def InterpolatePose(t):
  if (t < 5.0):
    # Duration of entry motion is set to 5 seconds.
    return InterpolatePoseEntry(t)
  elif (t >= 5.0) and (t < 6.0):
    # Wait for a second to grip the handle.
    return InterpolatePoseEntry(5.0)
  else:
    # Duration of the open motion is set to 5 seconds.
    return InterpolatePoseOpen((t - 6.0) / 5.0)

# Visualize our end-effector nominal trajectory. 
t_lst = np.linspace(0, 11, 30)
pose_lst = []
for t in t_lst:
  visualize_transform(str(t), InterpolatePose(t))
  pose_lst.append(InterpolatePose(t))
print('pose_lst: ', len(pose_lst))    

# Create gripper trajectory. 
gripper_t_lst = np.array([0., 5., 6., 11.])
gripper_knots = np.array([0.05, 0.05, 0., 0.]).reshape(1,4)
g_traj = PiecewisePolynomial.FirstOrderHold(gripper_t_lst, gripper_knots)

def CreateIiwaControllerPlant():
    """creates plant that includes only the robot and gripper, used for controllers."""
    print('CreateIiwaControllerPlant')
    robot_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf")
    gripper_sdf_path = FindResourceOrThrow(
        "drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_no_tip.sdf")
    sim_timestep = 1e-3
    plant_robot = MultibodyPlant(sim_timestep)
    parser = Parser(plant=plant_robot)
    parser.AddModelFromFile(robot_sdf_path)
    parser.AddModelFromFile(gripper_sdf_path)
    plant_robot.WeldFrames(
        A=plant_robot.world_frame(),
        B=plant_robot.GetFrameByName("iiwa_link_0"))
    plant_robot.WeldFrames(
        A=plant_robot.GetFrameByName("iiwa_link_7"),
        B=plant_robot.GetFrameByName("body"),
        X_AB=RigidTransform(RollPitchYaw(np.pi/2, 0, np.pi/2), np.array([0, 0, 0.114]))
    )
    plant_robot.mutable_gravity_field().set_gravity_vector([0, 0, 0])
    plant_robot.Finalize()

    link_frame_indices = []
    for i in range(8):
        link_frame_indices.append(
            plant_robot.GetFrameByName("iiwa_link_" + str(i)).index())

    return plant_robot, link_frame_indices

def BuildAndSimulateTrajectory(q_traj, g_traj):
  """Simulate trajectory for manipulation station.
  @param q_traj: Trajectory class used to initialize TrajectorySource for joints.
  @param g_traj: Trajectory class used to initialize TrajectorySource for gripper.
  """
  builder = DiagramBuilder()
  station = builder.AddSystem(ManipulationStation(time_step=1e-3))
  #station.SetupManipulationClassStation()
  #station.SetupClutterClearingStation()
  station.SetupNutStation()
  station.Finalize()

  q_traj_system = builder.AddSystem(TrajectorySource(q_traj))
  g_traj_system = builder.AddSystem(TrajectorySource(g_traj))

  meshcat = ConnectMeshcatVisualizer(builder,
          station.get_scene_graph(),
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,
          frames_to_draw={"gripper":{"body"}},
          zmq_url=zmq_url)
  
  builder.Connect(q_traj_system.get_output_port(),
                  station.GetInputPort("iiwa_position"))
  builder.Connect(g_traj_system.get_output_port(),
                  station.GetInputPort("wsg_position"))

  diagram = builder.Build()

  simulator = Simulator(diagram)
  simulator.set_target_realtime_rate(1.0)
  simulator.AdvanceTo(0.01)

  station_plant = station.get_multibody_plant()
  
  return simulator, station_plant

[2021-09-13 16:49:47.237] [console] [warning] Warning parsing file '/home/dmk0v/.cache/bazel/_bazel_dmk0v/53dec9d0c302c22abd6c7e6eb734c197/execroot/drake/bazel-out/k8-opt/bin/tutorials/door_opening.runfiles/drake/examples/manipulation_station/models/bolt.obj' : Material file [ ./bolt.obj.mtl ] not found in a path : /home/dmk0v/.cache/bazel/_bazel_dmk0v/53dec9d0c302c22abd6c7e6eb734c197/execroot/drake/bazel-out/k8-opt/bin/tutorials/door_opening.runfiles/drake/examples/manipulation_station/models/
Failed to load material file(s). Use default material.
material [ 'material_0' ] not found in .mtl
material [ 'material_1' ] not found in .mtl
material [ 'material_0' ] not found in .mtl
material [ 'material_1' ] not found in .mtl
material [ 'material_0' ] not found in .mtl
material [ 'material_2' ] not found in .mtl
material [ 'material_0' ] not found in .mtl
material [ 'material_2' ] not found in .mtl
material [ 'material_0' ] not found in .mtl

[2021-09-13 16:49:47.245] [console] [warning] Wa

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.
pose_lst:  30


/home/dmk0v/.cache/bazel/_bazel_dmk0v/53dec9d0c302c22abd6c7e6eb734c197/execroot/drake/bazel-out/k8-opt/bin/tutorials/door_opening.runfiles/drake/bindings/pydrake/systems/meshcat_visualizer.py:438: UserWarning: Non-existent frame gripper ignored
  warnings.warn(f"Non-existent frame {frame_id} ignored")


## Door Opening

In the lecture, we've seen an example of a robot manipulating a cylindrical object in the DARPA Robotics Challenge. By using a similar optimization-based inverse kinematics, We will solve a series of IK problems that can open a cupboard door. 

If you open up Meshcat at the link you got by running the first cell, you should be able to see that we provided you with setpoints ${}^W X^G(t)$ in the end-effector space, which gives us a nominal trajectory to reach and open the cupboard, which was computed by dividing the motion into following segments:

- $0 \leq t \leq 5$: The end-effector linearly interpolates between the initial pose at $t=0$, and the nominal grasp pose required to grip the cylinder (${}^W\mathbf{X}^H$ at $t=5$), while having the gripper open. 
- $5 \leq t \leq 6$: The end-effector stays still at ${}^W\mathbf{X}^H$, and the gripper is closed.
- $6 \leq t \leq 11$: The end-effector follows an arc of the handle as the door opens. The gripper remains closed.

Your job is to convert this end effector trajectory to a joint-space trajectory using optimization!



## Writing down the Optimization Problem

Before we code, let's think about what we want from this optimization problem by observing the below diagram. We will denote our decision variables as $q$ (joint angles of the arm), and the forward dynamics function as 

$${}^W\mathbf{X}^G = f(q)$$

(You may access the rotation and translation parts independently with ${}^W\mathbf{R}^G=f_R(q)$ and ${}^Wp^G=f_p(q)$). As written above, $H$ is the nominal frame which we've defined at the center of the handle.

<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/figures/exercises/door.png" width="700">

Recall one of the main ideas from the lecture: instead of having ${}^W\mathbf{X}^G=f(q)$ be constrained to exactly ${}^W\mathbf{X}^H$, the inverse kinematics problem can benefit a lot by allowing a set of ${}^W\mathbf{X}^G$ which makes sense for our problem. 





**Problem 8.1.a** [3pts]: Let us parametrize ${}^H X ^G$ with six degrees of freedom: xyz positions and roll-pitch-yaw rotation (i.e. rotations around the xyz axii, respectively) of frame $G$ with respect to frame $H$. Which three would you constrain to be exactly equal, and which three are you willing to provide some slack? Explain your reasoning.

(HINT: you should constrain two positions and one orientation).

**Problem 8.1.b** [2pts]: Write down an optimization problem to find $q$ with a joint centering cost on $q_{nom}$, and the constraints you wrote down in 8.1.a. You should write a equality constraint for the constrained degrees of freedom, and inequality constraints for the unconstrained one. Writing a constraint on some principal axis of orientation is fairly involved due to rotation formalisms, so you may fully constrain rotations for this problem. 

(HINT: The grading will allow some abuse of notation. For example, you can write down x component of $f_p(q)$ with $f_p(q)_x$, $f_p(q)[0]$, etc. But you should not be sloppy with frames). 

## Solving Optimization

**Now, it's your time to code!**

Below, you must implement `create_q_knots`, which accepts a list of key frames `pose_lst`, and converts them to joint coordinates. You will use Drake's `inverse_kinematics` package, for which we've provided simple wrappers called `AddOrientationConstraint` and `AddPositionConstraint`. 

For each keyframe `i`, our solution does the following:
- Implement constraints on `ik` using `AddOrientationConstraint` and `AddPositionConstraint`.
- Add a joint-centering cost on `q_nominal`.
- If `i==0`, Set initial guess to be nominal configuration using [`prog.SetInitialGuess`](https://drake.mit.edu/pydrake/pydrake.solvers.mathematicalprogram.html#pydrake.solvers.mathematicalprogram.MathematicalProgram.SetInitialGuess). Otherwise, set the initial guess to be the solution you obtained on the previous IK problem.

We will grade your solution by how far you were able to pull the door!


In [3]:
def create_q_knots(pose_lst):
  """Convert end-effector pose list to joint position list using series of 
  InverseKinematics problems. Note that q is 9-dimensional because the last 2 dimensions 
  contain gripper joints, but these should not matter to the constraints.
  @param: pose_lst (python list): post_lst[i] contains keyframe X_WG at index i.
  @return: q_knots (python_list): q_knots[i] contains IK solution that will give f(q_knots[i]) \approx pose_lst[i].
  """
  q_knots = []
  plant, _ = CreateIiwaControllerPlant()
  world_frame = plant.world_frame()
  gripper_frame = plant.GetFrameByName("body")
  q_nominal = np.array([ 0., 0.6, 0., -1.75, 0., 1., 0., 0., 0.]) # nominal joint for joint-centering.

  def AddOrientationConstraint(ik, R_WG, bounds):
    """Add orientation constraint to the ik problem. Implements an inequality 
    constraint where the axis-angle difference between f_R(q) and R_WG must be
    within bounds. Can be translated to:
    ik.prog().AddBoundingBoxConstraint(angle_diff(f_R(q), R_WG), -bounds, bounds)
    """
    ik.AddOrientationConstraint(
        frameAbar=world_frame, R_AbarA=R_WG, 
        frameBbar=gripper_frame, R_BbarB=RotationMatrix(),
        theta_bound=bounds
    )

  def AddPositionConstraint(ik, p_WG_lower, p_WG_upper):
    """Add position constraint to the ik problem. Implements an inequality
    constraint where f_p(q) must lie between p_WG_lower and p_WG_upper. Can be
    translated to 
    ik.prog().AddBoundingBoxConstraint(f_p(q), p_WG_lower, p_WG_upper)
    """
    ik.AddPositionConstraint(
        frameA=world_frame, frameB=gripper_frame, p_BQ=np.zeros(3),
        p_AQ_lower=p_WG_lower, p_AQ_upper=p_WG_upper)

  for i in range(len(pose_lst)):
    ik = inverse_kinematics.InverseKinematics(plant)
    q_variables = ik.q() # Get variables for MathematicalProgram
    prog = ik.prog() # Get MathematicalProgram
    print('i=', i)
    #### Modify here ###############################
    if 0==i:
      prog.SetInitialGuess(q_variables, q_nominal)
    else:
      prog.SetInitialGuess(q_variables, q_knots[-1])
    
    prog.AddCost(np.square(np.dot(q_variables, q_nominal)))
    theta = 0.15
    AddOrientationConstraint(ik, pose_lst[i].rotation(), theta)
    X_W_G = pose_lst[i]
    p_G_lower = np.array([-0.02, -0.03, -0.05])
    p_G_upper = np.array([0.04, 0.04, 0.22])

    p_W_G_lower = X_W_G.multiply(p_G_lower)
    p_W_G_upper =  X_W_G.multiply(p_G_upper)
    ps = np.hstack((p_W_G_lower[:, np.newaxis], p_W_G_upper[:, np.newaxis]))
    p_W_G_lower = np.min(ps, axis=1).flatten()
    p_W_G_upper = np.max(ps, axis=1).flatten()

    AddPositionConstraint(ik, p_W_G_lower, p_W_G_upper)

    ################################################
  
    result = Solve(prog)

    if not result.is_success():
      print('will quiet early')
      q_knots.append(q_knots[-1])
      continue
      #raise RuntimeError
      
    q_knots.append(result.GetSolution(q_variables))

  return q_knots

When you have implemented your function, you can run the cell below to initialize your simulation. Once everything loads up in Meshcat, run the next cell to run the full simulation!

In [4]:
q_knots = np.array(create_q_knots(pose_lst))
print(q_knots.shape)
print('t_lst', len(t_lst))
q_traj = PiecewisePolynomial.CubicShapePreserving(t_lst, q_knots[:, 0:7].T)
simulator, station_plant = BuildAndSimulateTrajectory(q_traj, g_traj)
simulator.AdvanceTo(0.1)


CreateIiwaControllerPlant
i= 0
i= 1
i= 2
i= 3
i= 4
i= 5
i= 6
i= 7
i= 8
i= 9
i= 10


<ipython-input-1-d765a3f8fdaa>:161: DrakeDeprecationWarning: Please use MultibodyPlant.WeldFrames(frame_on_parent_P=value1, frame_on_child_C=value2, X_PC=value3) instead. This variant will be removed after 2021-07-01.
  plant_robot.WeldFrames(


i= 11
i= 12
i= 13
i= 14
i= 15
i= 16
i= 17
i= 18
i= 19
i= 20
i= 21
i= 22
i= 23
i= 24
i= 25
i= 26
i= 27
i= 28
i= 29
(30, 9)
t_lst 30
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [5]:
simulator.AdvanceTo(11.0)
